In [1]:
import pandas as pd

In [104]:
!pip install pyLDAvis

    100% |████████████████████████████████| 1.6MB 1.8MB/s ta 0:00:01
  Stored in directory: /Users/TomasFox/Library/Caches/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [6]:
!pip install gensim

    100% |████████████████████████████████| 24.7MB 640kB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 340kB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 1.9MB/s ta 0:00:01
    100% |████████████████████████████████| 5.6MB 1.6MB/s ta 0:00:01    20% |██████▌                         | 1.1MB 2.0MB/s eta 0:00:03
    100% |████████████████████████████████| 71kB 2.4MB/s ta 0:00:01
  Stored in directory: /Users/TomasFox/Library/Caches/pip/wheels/5f/ea/fb/5b1a947b369724063b2617011f1540c44eb00e28c3d2ca8692
Successfully built smart-open


In [2]:
df = pd.read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [10]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [7]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [8]:
STOPWORDS = set(STOPWORDS).union(set(['amazon']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [9]:
tokenize("Hello World! This a test of the tokenization method")

['hello', 'world', 'test', 'tokenization', 'method']

In [30]:
path = '/Users/TomasFox/Downloads/DS-Unit-4-Sprint-1-NLP-master 3/module1-text-data/data'

In [43]:
df['tokens'] = df['reviews.text'].apply(tokenize)

In [45]:
id2word = corpora.Dictionary(df['tokens'])

In [56]:
id2word.doc2bow(tokenize('This is a sample message, serving as a test'))

[(415, 1), (433, 1), (3538, 1), (9119, 1)]

In [58]:
len(id2word.keys())

9620

In [62]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [69]:
corpus[1]

[(11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]

In [64]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=4
                  )

In [72]:
lda.print_topics()

[(0,
  '0.029*"case" + 0.024*"tablet" + 0.022*"comes" + 0.017*"app" + 0.012*"times" + 0.011*"kids" + 0.011*"children" + 0.011*"got" + 0.011*"use" + 0.010*"busy"'),
 (1,
  '0.092*"kindle" + 0.058*"easy" + 0.041*"use" + 0.026*"love" + 0.022*"read" + 0.022*"screen" + 0.018*"size" + 0.017*"light" + 0.017*"reading" + 0.014*"new"'),
 (2,
  '0.020*"new" + 0.013*"charging" + 0.012*"year" + 0.012*"charger" + 0.012*"device" + 0.011*"purchased" + 0.010*"warranty" + 0.010*"charge" + 0.009*"days" + 0.009*"problem"'),
 (3,
  '0.136*"tablet" + 0.041*"love" + 0.027*"kids" + 0.018*"ipad" + 0.017*"great" + 0.017*"screen" + 0.015*"tablets" + 0.011*"price" + 0.009*"good" + 0.009*"perfect"'),
 (4,
  '0.110*"batteries" + 0.051*"long" + 0.030*"work" + 0.027*"brand" + 0.021*"battery" + 0.018*"brands" + 0.017*"good" + 0.015*"time" + 0.014*"like" + 0.013*"use"'),
 (5,
  '0.045*"tablet" + 0.037*"great" + 0.028*"use" + 0.019*"reading" + 0.016*"love" + 0.015*"screen" + 0.014*"movies" + 0.013*"books" + 0.013*"easy"

In [80]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [81]:
words

[['case',
  'tablet',
  'comes',
  'app',
  'times',
  'kids',
  'children',
  'got',
  'use',
  'busy'],
 ['kindle',
  'easy',
  'use',
  'love',
  'read',
  'screen',
  'size',
  'light',
  'reading',
  'new'],
 ['new',
  'charging',
  'year',
  'charger',
  'device',
  'purchased',
  'warranty',
  'charge',
  'days',
  'problem'],
 ['tablet',
  'love',
  'kids',
  'ipad',
  'great',
  'screen',
  'tablets',
  'price',
  'good',
  'perfect'],
 ['batteries',
  'long',
  'work',
  'brand',
  'battery',
  'brands',
  'good',
  'time',
  'like',
  'use'],
 ['tablet',
  'great',
  'use',
  'reading',
  'love',
  'screen',
  'movies',
  'books',
  'easy',
  'good'],
 ['great',
  'price',
  'good',
  'product',
  'value',
  'works',
  'quality',
  'batteries',
  'beat',
  'work'],
 ['batteries',
  'box',
  'great',
  'buy',
  'need',
  'good',
  'nice',
  'price',
  'quality',
  'packaging'],
 ['tablet',
  'apps',
  'games',
  'play',
  'books',
  'use',
  'time',
  'kids',
  'like',
  'gre

In [83]:
#trying to understand code above 
lda.print_topics()[0]

(0,
 '0.029*"case" + 0.024*"tablet" + 0.022*"comes" + 0.017*"app" + 0.012*"times" + 0.011*"kids" + 0.011*"children" + 0.011*"got" + 0.011*"use" + 0.010*"busy"')

In [98]:
topics = [' '.join(t[0:10]) for t in words]

In [99]:
topics

['case tablet comes app times kids children got use busy',
 'kindle easy use love read screen size light reading new',
 'new charging year charger device purchased warranty charge days problem',
 'tablet love kids ipad great screen tablets price good perfect',
 'batteries long work brand battery brands good time like use',
 'tablet great use reading love screen movies books easy good',
 'great price good product value works quality batteries beat work',
 'batteries box great buy need good nice price quality packaging',
 'tablet apps games play books use time kids like great',
 'bought expected worked got like little wouldn ve said happy',
 'tablet kids great little good case apps product recommend features',
 'alexa sound tap echo like speaker device cover keyboard music',
 'buy best great deal good awesome black purchase friday sale',
 'loves old bought year tablet gift christmas love son daughter',
 'excellent product time battery good charge friendly life free working']

In [105]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [106]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.222174 -0.026952       1        1  13.516903
8      0.196801  0.019426       2        1  11.149769
13     0.223126  0.045250       3        1   9.413199
5      0.118812  0.117511       4        1   9.087641
1      0.126385 -0.014822       5        1   8.440866
6     -0.230555  0.220129       6        1   7.175814
7     -0.202682  0.006749       7        1   6.786618
10     0.086493  0.112020       8        1   5.511230
11    -0.036438 -0.103718       9        1   5.418077
3      0.075768  0.090996      10        1   5.102643
2      0.001601 -0.224182      11        1   4.480650
12    -0.085182  0.115693      12        1   4.271754
9     -0.051819 -0.191819      13        1   3.480350
14    -0.094064 -0.083266      14        1   3.227461
0      0.093928 -0.083016      15        1   2.937034, topic_info=     Category         Freq        Term        Total  loglift  logprob
19    Default  7114.000000   batteries  7114.000000  30.0000  30.0000
4195  Default  6844.000000      tablet  6844.000000  29.0000  29.0000
18    Default  4840.000000       price  4840.000000  28.0000  28.0000
22    Default  8902.000000       great  8902.000000  27.0000  27.0000
3155  Default  2973.000000      kindle  2973.000000  26.0000  26.0000
34    Default  5538.000000        good  5538.000000  25.0000  25.0000
2615  Default  2119.000000       loves  2119.000000  24.0000  24.0000
29    Default  2252.000000         buy  2252.000000  23.0000  23.0000
901   Default  2912.000000        easy  2912.000000  22.0000  22.0000
604   Default  2118.000000         old  2118.000000  21.0000  21.0000
234   Default  1415.000000        best  1415.000000  20.0000  20.0000
201   Default  2553.000000        kids  2553.000000  19.0000  19.0000
24    Default  2510.000000        long  2510.000000  18.0000  18.0000
107   Default  1775.000000        year  1775.000000  17.0000  17.0000
1058  Default  1015.000000        gift  1015.000000  16.0000  16.0000
173   Default  2037.000000     product  2037.000000  15.0000  15.0000
265   Default   734.000000   excellent   734.000000  14.0000  14.0000
92    Default  3395.000000        love  3395.000000  13.0000  13.0000
21    Default  1232.000000       brand  1232.000000  12.0000  12.0000
28    Default  3095.000000      bought  3095.000000  11.0000  11.0000
445   Default   979.000000       value   979.000000  10.0000  10.0000
771   Default  1674.000000       games  1674.000000   9.0000   9.0000
3171  Default  1420.000000       books  1420.000000   8.0000   8.0000
31    Default   856.000000   christmas   856.000000   7.0000   7.0000
2049  Default  1801.000000        apps  1801.000000   6.0000   6.0000
501   Default   775.000000        deal   775.000000   5.0000   5.0000
10    Default  2228.000000        work  2228.000000   4.0000   4.0000
1144  Default  1142.000000     reading  1142.000000   3.0000   3.0000
2244  Default  1644.000000      screen  1644.000000   2.0000   2.0000
8     Default  1305.000000     quality  1305.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
3010  Topic15    59.683369        ages    98.230240   3.0295  -5.0443
2749  Topic15    22.911369       shape    30.604403   3.2383  -6.0017
2796  Topic15    71.967171  protective   138.876587   2.8704  -4.8571
1142  Topic15    71.732521      member   140.149139   2.8580  -4.8604
304   Topic15   267.496521        case   851.307251   2.3701  -3.5442
2006  Topic15    57.279613     purpose   112.114517   2.8562  -5.0854
3652  Topic15    75.373055     dropped   168.067627   2.7259  -4.8109
799   Topic15    66.242218   installed   143.875000   2.7521  -4.9400
3627  Topic15   157.601318         app   527.003845   2.3206  -4.0733
6181  Topic15    46.625607      bumper    87.490326   2.8984  -5.2912
4730  Topic15    39.425713       users    72.754822   2.9151  -5.4589
175   Topic15   108.598923   

In [110]:
distro = [lda[d] for d in corpus]

In [115]:
distro[2]

[(0, 0.01666701),
 (1, 0.016667021),
 (2, 0.016667014),
 (3, 0.016667012),
 (4, 0.3152312),
 (5, 0.016667014),
 (6, 0.22614187),
 (7, 0.01666702),
 (8, 0.016667008),
 (9, 0.01666703),
 (10, 0.01666701),
 (11, 0.016667005),
 (12, 0.25862274),
 (13, 0.016667025),
 (14, 0.01666701)]

In [108]:
def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [116]:
new_distro[0]

{0: 0,
 1: 0,
 2: 0.2439354,
 3: 0,
 4: 0,
 5: 0,
 6: 0.19723463,
 7: 0.49212295,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0}

In [119]:
df = pd.DataFrame.from_records(new_distro)
df.columns = topics

In [120]:
df

,case tablet comes app times kids children got use busy,kindle easy use love read screen size light reading new,new charging year charger device purchased warranty charge days problem,tablet love kids ipad great screen tablets price good perfect,batteries long work brand battery brands good time like use,tablet great use reading love screen movies books easy good,great price good product value works quality batteries beat work,batteries box great buy need good nice price quality packaging,tablet apps games play books use time kids like great,bought expected worked got like little wouldn ve said happy,tablet kids great little good case apps product recommend features,alexa sound tap echo like speaker device cover keyboard music,buy best great deal good awesome black purchase friday sale,loves old bought year tablet gift christmas love son daughter,excellent product time battery good charge friendly life free working
0,0.000000,0.000000,0.243935,0.000000,0.000000,0.000000,0.197235,0.492123,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.011112,0.011112,0.011112,0.011112,0.490093,0.011112,0.011112,0.365457,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112
2,0.016667,0.016667,0.016667,0.016667,0.315132,0.016667,0.225731,0.016667,0.016667,0.016667,0.016667,0.016667,0.259132,0.016667,0.016667
3,0.011111,0.011111,0.011111,0.011111,0.844443,0.011111,0.011112,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111
4,0.011111,0.011111,0.011111,0.011111,0.538949,0.011111,0.316606,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111
5,0.000000,0.000000,0.000000,0.000000,0.556585,0.000000,0.000000,0.334428,0.000000,0.000000,0.000000,0.000000,0.000000,0.072620,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.866566,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.461016,0.415165,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.906662,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.915148,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
